In [1]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from captum import attr
from attrbench import models, attribution
from tqdm import tqdm
import torch

In [3]:
class TestCifar(Dataset):
    def __init__(self):
        self.cifar10 = datasets.CIFAR10(root="../data/CIFAR10/", download=False, train=False, transform=transforms.ToTensor())
    
    def __getitem__(self, index):
        data, target = self.cifar10[index]
        return data, target, index
    
    def __len__(self):
        return len(self.cifar10)

In [4]:
ds = TestCifar()
dl = DataLoader(ds, batch_size=256, shuffle=False, drop_last=True, num_workers=4)

In [5]:
device = "cuda"
model = models.Resnet(version="resnet18", output_logits=True, num_classes=10,
                      params_loc="../data/models/CIFAR10/resnet18.pt")
model.to(device)
model.eval()
pass

In [6]:
#saliency = attribution.GradCAM(model, model.get_last_conv_layer(), (32, 32))
saliency = attribution.GuidedGradCAM(model, model.get_last_conv_layer(), (32, 32))
#saliency = attribution.OldGuidedGradCAM(model, model.get_last_conv_layer())

In [7]:
bad_idxs = []
for batch_idx, (data, target, idx) in enumerate(tqdm(dl)):
    data = data.to(device)
    target = target.to(device)
    attrs = saliency(data, target)
    
    abs_attrs = torch.abs(attrs.flatten(1))
    max_abs_per_img = torch.max(abs_attrs, dim=1)[0]
    if torch.any(max_abs_per_img == 0):
        bad_idxs.append(idx[torch.where(max_abs_per_img == 0)])
if len(bad_idxs) > 0:
    bad_idxs = torch.cat(bad_idxs)

  0%|          | 0/39 [00:00<?, ?it/s]/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.7/site-packages/captum/attr/_utils/gradient.py:33: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.7/site-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.7/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:56: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


  3%|▎         | 1/39 [00:00<00:34,  1.09it/s]

  5%|▌         | 2/39 [00:01<00:31,  1.16it/s]

  8%|▊         | 3/39 [00:02<00:29,  1.21it/s]

 10%|█         | 4/39 [00:03<00:28,  1.24it/s]

...

 90%|████████▉ | 35/39 [00:26<00:03,  1.33it/s]

 92%|█████████▏| 36/39 [00:27<00:02,  1.33it/s]

 95%|█████████▍| 37/39 [00:27<00:01,  1.33it/s]

 97%|█████████▋| 38/39 [00:28<00:00,  1.33it/s]

100%|██████████| 39/39 [00:29<00:00,  1.32it/s]

In [8]:
len(bad_idxs)

6624